In [3]:
import os

def get_image_paths_and_labels(root_dir):
    image_paths = []
    labels = []
    for subset in ['train', 'test', 'val']:
        for label in ['Paper', 'Rock', 'Scissors']:
            folder_path = os.path.join(root_dir, f'RPS_{subset}', f'{label}_{subset}')
            for image_file in os.listdir(folder_path):
                if image_file.endswith('.png'):
                    image_paths.append(os.path.join(folder_path, image_file))
                    labels.append(label)
    return image_paths, labels

# Example usage:
root_directory = "E:/NYCU/Signal & Image Process Data/Police txt & RPS Dataset/RPS dataset"
train_paths, train_labels = get_image_paths_and_labels(root_directory)
print("Train set size:", len(train_paths))

# Data Loader (basic implementation)
class DataLoader:
    def __init__(self, image_paths, labels):
        self.image_paths = image_paths
        self.labels = labels
        self.length = len(image_paths)
        self.current_index = 0

    def __len__(self):
        return self.length

    def __iter__(self):
        return self

    def __next__(self):
        if self.current_index < self.length:
            image_path = self.image_paths[self.current_index]
            label = self.labels[self.current_index]
            self.current_index += 1
            return image_path, label
        else:
            raise StopIteration

# Example usage of the DataLoader
train_loader = DataLoader(train_paths, train_labels)
for i, (image_path, label) in enumerate(train_loader):
    print(f"Image Path: {image_path}, Label: {label}")
    if i >= 500:  # Print the first 5 examples
        break


Train set size: 2100
Image Path: E:/NYCU/Signal & Image Process Data/Police txt & RPS Dataset/RPS dataset\RPS_train\Paper_train\04l5I8TqdzF9WDMJ.png, Label: Paper
Image Path: E:/NYCU/Signal & Image Process Data/Police txt & RPS Dataset/RPS dataset\RPS_train\Paper_train\0a3UtNzl5Ll3sq8K.png, Label: Paper
Image Path: E:/NYCU/Signal & Image Process Data/Police txt & RPS Dataset/RPS dataset\RPS_train\Paper_train\0cb6cVL8pkfi4wF6.png, Label: Paper
Image Path: E:/NYCU/Signal & Image Process Data/Police txt & RPS Dataset/RPS dataset\RPS_train\Paper_train\0eqArS2GgsBeqgSn.png, Label: Paper
Image Path: E:/NYCU/Signal & Image Process Data/Police txt & RPS Dataset/RPS dataset\RPS_train\Paper_train\0Og76sl5CJhbxWWx.png, Label: Paper
Image Path: E:/NYCU/Signal & Image Process Data/Police txt & RPS Dataset/RPS dataset\RPS_train\Paper_train\0RA9rcrv8iVvuDDU.png, Label: Paper
Image Path: E:/NYCU/Signal & Image Process Data/Police txt & RPS Dataset/RPS dataset\RPS_train\Paper_train\0t08v9bAaA0mXGCQ.png

In [5]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from tqdm import tqdm

# Function to get image paths and labels
def get_image_paths_and_labels(root_dir):
    image_paths = []
    labels = []
    for subset in ['train', 'test', 'val']:
        for label in ['Paper', 'Rock', 'Scissors']:
            folder_path = os.path.join(root_dir, f'RPS_{subset}', f'{label}_{subset}')
            for image_file in os.listdir(folder_path):
                if image_file.endswith('.png'):
                    image_paths.append(os.path.join(folder_path, image_file))
                    labels.append(label)
    return image_paths, labels

# Define transformations for data augmentation and normalization
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to a fixed size
    transforms.ToTensor(),           # Convert images to PyTorch tensors
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize images
])

# Load datasets and create data loaders
root_directory = "E:/NYCU/Signal & Image Process Data/Police txt & RPS Dataset/RPS dataset"
train_paths, train_labels = get_image_paths_and_labels(root_directory)
train_dataset = ImageFolder(root=os.path.join(root_directory, 'RPS_train'), transform=transform)
val_dataset = ImageFolder(root=os.path.join(root_directory, 'RPS_val'), transform=transform)
test_dataset = ImageFolder(root=os.path.join(root_directory, 'RPS_test'), transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)

# Define the model
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 3)  # 3 output classes for Rock, Paper, Scissors

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Train the model
num_epochs = 5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in tqdm(train_loader, desc=f'Epoch {epoch + 1}/{num_epochs}', leave=False):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    epoch_loss = running_loss / len(train_dataset)
    print(f'Training Loss: {epoch_loss:.4f}')

    # Validate the model
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    val_accuracy = correct / total
    print(f'Validation Accuracy: {val_accuracy:.4f}')

# Evaluate the model on test data
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
test_accuracy = correct / total
print(f'Test Accuracy: {test_accuracy:.4f}')


C:\Users\SHAKEEL\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\SHAKEEL\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\SHAKEEL/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:14<00:00, 3.20MB/s]


Training Loss: 0.3570
Validation Accuracy: 0.9810


Training Loss: 0.0369
Validation Accuracy: 1.0000


Training Loss: 0.0170
Validation Accuracy: 1.0000


Training Loss: 0.0117
Validation Accuracy: 1.0000


Training Loss: 0.0084
Validation Accuracy: 1.0000
Test Accuracy: 1.0000
